POSITIVI IN ITALIA

In [17]:
!wget https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv 

--2021-02-08 08:52:07--  https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40957 (40K) [text/plain]
Saving to: ‘dpc-covid19-ita-andamento-nazionale.csv.1’

dpc-covid19-ita-and 100%[===================>]  40.00K  --.-KB/s    in 0.004s  

2021-02-08 08:52:08 (10.2 MB/s) - ‘dpc-covid19-ita-andamento-nazionale.csv.1’ saved [40957/40957]



In [18]:
!pip3 install plotly

In [19]:
import os
import plotly.graph_objects as go

GRAFICO TOTALE CASI

In [20]:
#VITERBO (ITALIA)

covid19_viterbo = open ("dpc-covid19-ita-andamento-nazionale.csv", "r").readlines()

covid19_viterbo = [i.split(",") for i in covid19_viterbo]

data = [i[0] for i in covid19_viterbo]
totale_casi = [i[9] for i in covid19_viterbo]

print ("TOTALE CASI:", totale_casi[-1])

print ("TOTALE CASI OGGI: ", data[-1][:10], ": ", int(totale_casi[-1]) - int(totale_casi[-2]))

fig = go.Figure(data=[
    go.Bar(name='data', x=data, y=totale_casi)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

TOTALE CASI: 2118441
TOTALE CASI OGGI:  2021-02-07 :  11380


GRAFICO TOTALE CASI ODIERNI

In [21]:
x = totale_casi[1:]

totale_casi_odierni = [int(x[n])-int(x[n-1]) for n in range(1,len(x))]

fig = go.Figure(data=[
    go.Bar(name='data', x=data, y=totale_casi_odierni)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [22]:
!pip3 install pystan
!pip3 install fbprophet

In [23]:
import pandas as pd
import fbprophet
from fbprophet import Prophet

FB PROPHET PREDICTION

In [24]:
starting_data = "2020-10-01"
predicted_days = 7

for i in data:
    if starting_data in i:
        observation_number = data.index(i)
        observation_number = len(data) - observation_number
        
variable_0 = "data"
variable_1 = data[-(observation_number):]
variable_2 = "totale_casi_odierni"
variable_3 = totale_casi_odierni[-(observation_number):]

df = pd.DataFrame(
    {variable_0: variable_1,
     variable_2: variable_3
    })

df.columns = ["ds","y"]

model = Prophet(yearly_seasonality=False,weekly_seasonality=False, daily_seasonality=False)
model.fit(df)

future = model.make_future_dataframe(periods=predicted_days) #, freq='D'

pred = model.predict(future)

pred[-(predicted_days):]

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
130,2021-02-08 17:00:00,17258.832231,10014.387554,24201.903303,17258.832231,17258.832231,0.0,0.0,0.0,0.0,0.0,0.0,17258.832231
131,2021-02-09 17:00:00,17184.453963,9589.538777,24115.483299,17180.152420,17189.936096,0.0,0.0,0.0,0.0,0.0,0.0,17184.453963
132,2021-02-10 17:00:00,17110.075695,9960.434236,24502.358044,17095.762469,17128.223102,0.0,0.0,0.0,0.0,0.0,0.0,17110.075695
133,2021-02-11 17:00:00,17035.697426,9535.385266,24135.622692,17004.893879,17071.946834,0.0,0.0,0.0,0.0,0.0,0.0,17035.697426
134,2021-02-12 17:00:00,16961.319158,9885.495023,24449.884689,16913.716633,17021.018958,0.0,0.0,0.0,0.0,0.0,0.0,16961.319158
135,2021-02-13 17:00:00,16886.940890,9380.681160,23806.261982,16816.966691,16970.096030,0.0,0.0,0.0,0.0,0.0,0.0,16886.940890
136,2021-02-14 17:00:00,16812.562622,9538.432977,23709.067876,16721.743032,16925.175769,0.0,0.0,0.0,0.0,0.0,0.0,16812.562622


In [25]:
date_pred = list(pred['ds'])
date_pred = (date_pred[-(predicted_days):])

yhat = list(pred['yhat'])
yhat = (yhat[-(predicted_days):])
yhat = [round(i,2) for i in yhat]

yhat_lower = list(pred['yhat_lower'])
yhat_lower = (yhat_lower[-(predicted_days):])
yhat_lower = [round(i,2) for i in yhat_lower]

yhat_upper = list(pred['yhat_upper'])
yhat_upper = (yhat_upper[-(predicted_days):])
yhat_upper = [round(i,2) for i in yhat_upper]

date_yhat = zip(date_pred, yhat_lower, yhat, yhat_upper)
date_yhat = (str(list(date_yhat))).replace (",","\n")
print (date_yhat)

fig = go.Figure(data=[
    go.Bar(name='yhat_lower', x=date_pred, y=yhat_lower),
    go.Bar(name='yhat', x=date_pred, y=yhat),
    go.Bar(name='yhat_upper', x=date_pred, y=yhat_upper)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

[(Timestamp('2021-02-08 17:00:00')
 10014.39
 17258.83
 24201.9)
 (Timestamp('2021-02-09 17:00:00')
 9589.54
 17184.45
 24115.48)
 (Timestamp('2021-02-10 17:00:00')
 9960.43
 17110.08
 24502.36)
 (Timestamp('2021-02-11 17:00:00')
 9535.39
 17035.7
 24135.62)
 (Timestamp('2021-02-12 17:00:00')
 9885.5
 16961.32
 24449.88)
 (Timestamp('2021-02-13 17:00:00')
 9380.68
 16886.94
 23806.26)
 (Timestamp('2021-02-14 17:00:00')
 9538.43
 16812.56
 23709.07)]
